In [3]:
import sqlite3
import os

data_dir = "data/distributions"
info_name = 'inter_arrival_time'
table_name = 'inter_arrival'
# conn = sqlite3.connect(os.path.join(data_dir, f'cache_background_{info_name}_db_1.000000_flowmult_11.850000_intermult_0.db'))
conn = sqlite3.connect(os.path.join(data_dir, f'web_bursty_{info_name}_db_1.000000_flowmult_0.110000_intermult_40_flow_per_incast_20000_flow_size_0.db'))
cursor = conn.cursor()

# Fetch table structure
cursor.execute(f"PRAGMA table_info({table_name});")
columns = cursor.fetchall()
print(f"Columns in {table_name}:", columns)

# Fetch and display sample data
cursor.execute(f"SELECT * FROM {table_name} LIMIT 10;")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the connection
conn.close()

Columns in inter_arrival: []


OperationalError: no such table: inter_arrival

In [2]:
import pandas as pd
import numpy as np

db_paths = {
    'flow_ids': '/home/ubuntu/p4burst/data/distributions/cache_background_flow_ids_db_1.000000_flowmult_11.850000_intermult_0.db',
    'flow_size': '/home/ubuntu/p4burst/data/distributions/cache_background_flow_size_db_1.000000_flowmult_11.850000_intermult_0.db',
    'inter_arrival_time': '/home/ubuntu/p4burst/data/distributions/cache_background_inter_arrival_time_db_1.000000_flowmult_11.850000_intermult_0.db',
    'server_idx': '/home/ubuntu/p4burst/data/distributions/cache_background_server_idx_db_1.000000_flowmult_11.850000_intermult_0.db'
}

# Dictionary to store dataframes
dataframes = {}

# Process each database file
for aspect, db_path in db_paths.items():
    # Connect to the database
    conn = sqlite3.connect(db_path)
    
    # Extract the table name (assuming a single table in each db)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_name = cursor.fetchone()[0]
    print(table_name)
    
    # Load the table into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    dataframes[aspect] = df

    # Display first few rows
    print(f"{aspect} data:")
    print(df.head())
    
    # Close the connection
    conn.close()

    # Retrieve the specific column for this server's host ID
    # if 'server0app1' in df.columns:
    #     print(df['server0app1'].replace('', np.nan).dropna().tolist())
    # else:
    #     print('NO')

flow_ids
flow_ids data:
  server0app1 server1app1 server2app1 server3app1 server4app1 server5app1  \
0         100         108        1017        1030        1036        1046   
1         101         109        1018        1031        1037        1047   
2         102        1010        1019        1032        1038        1048   
3         103        1011        1020        1033        1039        1049   
4         104        1012        1021        1034        1040        1050   

  server6app1 server7app1 server8app1 server9app1  ... server310app1  \
0        1058        1077        1088        1095  ...        102976   
1        1059        1078        1089        1096  ...        102977   
2        1060        1079        1090        1097  ...        102978   
3        1061        1080        1091        1098  ...        102979   
4        1062        1081        1092        1099  ...        102980   

  server311app1 server312app1 server313app1 server314app1 server315app1  \
0    

In [3]:
# import numpy as np
# # Stack the 'server_idx' DataFrame, convert values to integers, handle missing values, and sort them
# unique_values = (
#     dataframes['server_idx']
#     .replace('', np.nan)             # Replace empty strings with NaN
#     .stack()                         # Stack all columns into a single Series
#     .dropna()                        # Remove any NaN values if present
#     .astype(int)                     # Convert values to integers
#     .unique()                        # Get unique values
# )
# sorted_unique_values = sorted(unique_values)  # Sort the unique values
# sorted_unique_values

In [4]:
# test_name = 'data/distributions/cache_background_server_idx_db_1.000000_flowmult_11.850000_intermult_0.db'
# conn = sqlite3.connect(test_name)
# cursor = conn.cursor()
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# table_name = cursor.fetchone()[0]
# print(table_name)

## Cache Background

In [45]:
import sqlite3
import pandas as pd
import random

# Define the target number of servers (K)
K = 320

# Define the path to save the new modified databases
output_dir = '/home/ubuntu/p4burst/data/distributions/'

# Load the original databases
db_paths = {
    'flow_ids': '/home/ubuntu/p4burst/data/distributions/cache_background_flow_ids_db_1.000000_flowmult_11.850000_intermult_0.db',
    'flow_size': '/home/ubuntu/p4burst/data/distributions/cache_background_flow_size_db_1.000000_flowmult_11.850000_intermult_0.db',
    'inter_arrival_time': '/home/ubuntu/p4burst/data/distributions/cache_background_inter_arrival_time_db_1.000000_flowmult_11.850000_intermult_0.db',
    'server_idx': '/home/ubuntu/p4burst/data/distributions/cache_background_server_idx_db_1.000000_flowmult_11.850000_intermult_0.db'
}

# Dictionary to store dataframes
dataframes = {}

# Process each database file
for aspect, db_path in db_paths.items():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_name = cursor.fetchone()[0]
    
    # Load the table into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    conn.close()  # Close the database connection
    
    # Keep only the first K columns
    df = df.iloc[:, :K]
    
    # If modifying server_idx, reassign server targets within K servers
    if aspect == "server_idx":
        for sender, col in enumerate(df.columns):
            new_targets = []
            for dst_server_idx in range(len(df[col])):
                if df[col][dst_server_idx] == '':
                    new_targets.append('')
                    continue
                # Exclude the current server as a target
                targets = list(range(K))
                targets.remove(sender)  # Modulo to prevent out-of-bounds
                new_targets.append(random.choice(targets))
            df[col] = new_targets  # Assign new target servers

    # Store the modified DataFrame in the dictionary
    dataframes[aspect] = df

# Save the modified DataFrames with the new file naming convention
for aspect, df in dataframes.items():
    output_db_path = f"{output_dir}cache_background_{aspect}_db_1.000000_flowmult_11.850000_intermult_0_{K}_servers.db"
    # Save to a new SQLite database
    conn = sqlite3.connect(output_db_path)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

print(f"New datasets saved in {output_dir} with filenames prefixed by '{K}_servers'")


New datasets saved in /home/ubuntu/p4burst/data/distributions/ with filenames prefixed by '320_servers'


In [46]:
import pandas as pd
import numpy as np

db_paths = dict()
for aspect, df in dataframes.items():
    db_paths[aspect] = f"{output_dir}cache_background_{aspect}_db_1.000000_flowmult_11.850000_intermult_0_{K}_servers.db"

# Dictionary to store dataframes
dataframes = {}

# Process each database file
for aspect, db_path in db_paths.items():
    # Connect to the database
    conn = sqlite3.connect(db_path)
    
    # Extract the table name (assuming a single table in each db)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_name = cursor.fetchone()[0]
    print(table_name)
    
    # Load the table into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    dataframes[aspect] = df

    # Display first few rows
    print(f"{aspect} data:")
    print(df.head())
    
    # Close the connection
    conn.close()

server_idx
flow_ids data:
  server0app1 server1app1 server2app1 server3app1 server4app1 server5app1  \
0         100         108        1017        1030        1036        1046   
1         101         109        1018        1031        1037        1047   
2         102        1010        1019        1032        1038        1048   
3         103        1011        1020        1033        1039        1049   
4         104        1012        1021        1034        1040        1050   

  server6app1 server7app1 server8app1 server9app1  ... server310app1  \
0        1058        1077        1088        1095  ...        102976   
1        1059        1078        1089        1096  ...        102977   
2        1060        1079        1090        1097  ...        102978   
3        1061        1080        1091        1098  ...        102979   
4        1062        1081        1092        1099  ...        102980   

  server311app1 server312app1 server313app1 server314app1 server315app1  \
0  

In [47]:
dataframes['server_idx']

,server0app1,server1app1,server2app1,server3app1,server4app1,server5app1,server6app1,server7app1,server8app1,server9app1,...,server310app1,server311app1,server312app1,server313app1,server314app1,server315app1,server316app1,server317app1,server318app1,server319app1
0,125,200,92,86,294,215,287,240,33,32,...,233,102,193,230,207,319,157,184,297,26
1,197,60,285,54,228,173,239,238,274,165,...,169,281,114,138,255,46,61,143,132,118
2,11,28,135,236,26,246,42,70,63,307,...,126,99,146,233,186,281,166,256,296,218
3,100,149,228,319,130,309,54,150,221,116,...,38,230,278,314,122,87,124,177,276,213
4,65,47,168,173,1,72,212,80,58,239,...,246,275,281,296,152,138,279,46,58,215
5,77,54,11,190,163,221,98,222,58,210,...,152,46,,96,267,213,211,78,112,218
6,109,226,279,,237,226,70,136,221,277,...,74,91,,171,,79,,92,202,183
7,45,178,82,,165,276,279,68,,,...,,230,,58,,114,,63,35,315
8,,274,53,,274,182,283,63,,,...,,237,,20,,99,,142,96,
9,,,211,,286,201,249,250,,,...,,,,15,,,,309,84,


In [48]:
flow_data = []
db_path = 'data/distributions'

# Determine the column name based on the host ID
server_column = f"server2app1"  # Convert host_id hX to serverXapp0

# Construct the file paths based on the specified format
db_params = {
    "server_indices": "server_idx",
    "inter_arrival_times": "inter_arrival_time",
    "flow_sizes": "flow_size",
    "flow_ids": "flow_ids"
}

db_tables = {
    "server_indices": "server_idx",
    "inter_arrival_times": "inter_arrival",
    "flow_sizes": "flow_size",
    "flow_ids": "flow_ids"
}

# Load each parameter from its corresponding database file
data = {}
for (param_name, db_name), db_col in zip(db_params.items(), db_tables.values()):
    file_path = f"cache_background_{db_name}_db_" \
                f"{float(1):.6f}_flowmult_" \
                f"{float(11.85):.6f}_intermult_0_" \
                f"4_servers.db"
    try:
        conn = sqlite3.connect(os.path.join(db_path, file_path))
        print(file_path)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        table_name = cursor.fetchone()[0]

        # Retrieve only the specific column for this server's host id
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        conn.close()
    except:
        raise ValueError(f"Error loading data from {file_path}")

    # Retrieve the specific column for this server's host ID
    if server_column in df.columns:
        data[param_name] = df[server_column].replace('', np.nan).dropna().tolist()
    else:
        raise ValueError(f"Column {server_column} not found in {file_path}")
    
# Combine the data from each database into flow data entries
for flow_id, (inter_arrival_time, flow_size, server_idx) in enumerate(
        zip(data["inter_arrival_times"], data["flow_sizes"], data["server_indices"])):
    flow_data.append({
        'flow_id': flow_id,
        'inter_arrival_time': inter_arrival_time,
        'flow_size': flow_size,
        'server_idx': server_idx  # Destination server
    })

cache_background_server_idx_db_1.000000_flowmult_11.850000_intermult_0_4_servers.db
cache_background_inter_arrival_time_db_1.000000_flowmult_11.850000_intermult_0_4_servers.db
cache_background_flow_size_db_1.000000_flowmult_11.850000_intermult_0_4_servers.db
cache_background_flow_ids_db_1.000000_flowmult_11.850000_intermult_0_4_servers.db


## Web Bursty

In [14]:
import sqlite3
import pandas as pd
import random

# Define the target number of servers (K)
K = 100

# Define the path to save the new modified databases
output_dir = '/home/ubuntu/p4burst/data/distributions/'

# Load the original databases
db_paths = {
    'flow_ids': '/home/ubuntu/p4burst/data/distributions/web_bursty_flow_ids_db_1.000000_flowmult_0.110000_intermult_40_flows_per_incast_20000_incast_flow_size_0.db',
    'query_ids': '/home/ubuntu/p4burst/data/distributions/web_bursty_query_ids_db_1.000000_flowmult_0.110000_intermult_40_flows_per_incast_20000_incast_flow_size_0.db',
    'inter_arrival_time': '/home/ubuntu/p4burst/data/distributions/web_bursty_inter_arrival_time_db_1.000000_flowmult_0.110000_intermult_40_flows_per_incast_20000_incast_flow_size_0.db',
    'server_idx': '/home/ubuntu/p4burst/data/distributions/web_bursty_server_idx_db_1.000000_flowmult_0.110000_intermult_40_flows_per_incast_20000_incast_flow_size_0.db'
}

# Dictionary to store dataframes
dataframes = {}

# Process each database file
for aspect, db_path in db_paths.items():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_name = cursor.fetchone()[0]
    
    # Load the table into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    conn.close()  # Close the database connection
    
    # Keep only the first K columns
    df = df.iloc[:, :K]
    
    # If modifying server_idx, reassign server targets within K servers
    if aspect == "server_idx":
        for sender, col in enumerate(df.columns):
            new_targets = []
            for dst_server_idx in range(len(df[col])):
                if df[col][dst_server_idx] == '':
                    new_targets.append('')
                    continue
                # Exclude the current server as a target
                targets = list(range(K))
                targets.remove(sender)  # Modulo to prevent out-of-bounds
                new_targets.append(random.choice(targets))
            df[col] = new_targets  # Assign new target servers

    # Store the modified DataFrame in the dictionary
    dataframes[aspect] = df

# Save the modified DataFrames with the new file naming convention
for aspect, df in dataframes.items():
    output_db_path = f"{output_dir}web_bursty_{aspect}_db_1.000000_flowmult_0.110000_intermult_40_flows_per_incast_20000_incast_flow_size_0_{K}_servers.db"
    # Save to a new SQLite database
    conn = sqlite3.connect(output_db_path)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

print(f"New datasets saved in {output_dir} with filenames prefixed by '{K}_servers'")


New datasets saved in /home/ubuntu/p4burst/data/distributions/ with filenames prefixed by '100_servers'


In [15]:
import pandas as pd
import numpy as np

db_paths = dict()
for aspect, df in dataframes.items():
    db_paths[aspect] = f"{output_dir}web_bursty_{aspect}_db_1.000000_flowmult_0.110000_intermult_40_flows_per_incast_20000_incast_flow_size_0_{K}_servers.db"

# Dictionary to store dataframes
dataframes = {}

# Process each database file
for aspect, db_path in db_paths.items():
    # Connect to the database
    conn = sqlite3.connect(db_path)
    
    # Extract the table name (assuming a single table in each db)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_name = cursor.fetchone()[0]
    print(table_name)
    
    # Load the table into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    dataframes[aspect] = df

    # Display first few rows
    print(f"{aspect} data:")
    print(df.head())
    
    # Close the connection
    conn.close()

server_idx
flow_ids data:
  server0app2 server1app2 server2app2 server3app2 server4app2 server5app2  \
0      114680      112280      115840      117600     1114480       11600   
1      114681      112281      115841      117601     1114481       11601   
2      114682      112282      115842      117602     1114482       11602   
3      114683      112283      115843      117603     1114483       11603   
4      114684      112284      115844      117604     1114484       11604   

  server6app2 server7app2 server8app2 server9app2  ... server90app2  \
0      112640     1112960      116000       11840  ...       113480   
1      112641     1112961      116001       11841  ...       113481   
2      112642     1112962      116002       11842  ...       113482   
3      112643     1112963      116003       11843  ...       113483   
4      112644     1112964      116004       11844  ...       113484   

  server91app2 server92app2 server93app2 server94app2 server95app2  \
0       119280

In [17]:
dataframes['inter_arrival_time']

,server0app2,server1app2,server2app2,server3app2,server4app2,server5app2,server6app2,server7app2,server8app2,server9app2,...,server90app2,server91app2,server92app2,server93app2,server94app2,server95app2,server96app2,server97app2,server98app2,server99app2
0,0.25262369834886333,0.2262767910852821,0.2610408776641066,0.28411987891507073,0.36200309618728915,0.20864475102084912,0.23093178082838628,0.33902433319872194,0.26162231645374645,0.2110736266236331,...,0.23889463085944584,0.29885768433878007,0.29442427669074217,0.29013401469900046,0.36928854836368924,0.3023301454319964,0.22708342129813458,0.21855158374151568,0.4743717571505177,0.36361422952817607
1,0.25262369834886333,0.2262767910852821,0.2610408776641066,0.28411987891507073,0.36200309618728915,0.20864475102084912,0.23093178082838628,0.33902433319872194,0.26162231645374645,0.2110736266236331,...,0.23889463085944584,0.29885768433878007,0.29442427669074217,0.29013401469900046,0.36928854836368924,0.3023301454319964,0.22708342129813458,0.21855158374151568,0.4743717571505177,0.36361422952817607
2,0.25262369834886333,0.2262767910852821,0.2610408776641066,0.28411987891507073,0.36200309618728915,0.20864475102084912,0.23093178082838628,0.33902433319872194,0.26162231645374645,0.2110736266236331,...,0.23889463085944584,0.29885768433878007,0.29442427669074217,0.29013401469900046,0.36928854836368924,0.3023301454319964,0.22708342129813458,0.21855158374151568,0.4743717571505177,0.36361422952817607
3,0.25262369834886333,0.2262767910852821,0.2610408776641066,0.28411987891507073,0.36200309618728915,0.20864475102084912,0.23093178082838628,0.33902433319872194,0.26162231645374645,0.2110736266236331,...,0.23889463085944584,0.29885768433878007,0.29442427669074217,0.29013401469900046,0.36928854836368924,0.3023301454319964,0.22708342129813458,0.21855158374151568,0.4743717571505177,0.36361422952817607
4,0.25262369834886333,0.2262767910852821,0.2610408776641066,0.28411987891507073,0.36200309618728915,0.20864475102084912,0.23093178082838628,0.33902433319872194,0.26162231645374645,0.2110736266236331,...,0.23889463085944584,0.29885768433878007,0.29442427669074217,0.29013401469900046,0.36928854836368924,0.3023301454319964,0.22708342129813458,0.21855158374151568,0.4743717571505177,0.36361422952817607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,,,,,,,,,,,...,,,,,,,,,,
596,,,,,,,,,,,...,,,,,,,,,,
597,,,,,,,,,,,...,,,,,,,,,,
598,,,,,,,,,,,...,,,,,,,,,,


## Test background load computation

In [58]:
import sqlite3 
import pandas as pd 

def compute_background_load(db_params, n_hosts, host_to_switch_bw=10, switch_to_switch_bw=40, iat_scale_factor=1):
    """Compute background load percentage based on DB flow parameters."""
    # Load the database files
    db_paths = {
        aspect: f"data/distributions/cache_background_{db_name}_db_{1:.6f}_flowmult_{11.85:.6f}_intermult_0_{n_hosts}_servers.db"
        for aspect, db_name in db_params.items()
    }

    dataframes = {}

    # Process each database file
    for aspect, db_path in db_paths.items():
        # Connect to the database
        conn = sqlite3.connect(db_path)
        
        # Extract the table name (assuming a single table in each db)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        table_name = cursor.fetchone()[0]
        # print(table_name)
        
        # Load the table into a DataFrame
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        dataframes[aspect] = df

        # Display first few rows
        # print(f"{aspect} data:")
        # print(df.head())
        
        # Close the connection
        conn.close()

    # Retrieve flow sizes and inter-arrival times
    flow_sizes_df = dataframes['flow_sizes']
    iat_df = dataframes['inter_arrival_times']

    total_traffic_rate_mbps = 0  # Total traffic rate in Mbps
    
    for server in flow_sizes_df.columns:
        flow_sizes = flow_sizes_df[server]
        iats = iat_df[server]

        # Compute traffic rate for each flow
        for flow_size, iat in zip(flow_sizes, iats):
            if flow_size and iat:  # Skip empty values
                if float(iat) > 0:  # Avoid division by zero
                    # Convert flow size to Mbps and adjust by multipliers
                    flow_size_bits = (float(flow_size) * 8)
                    traffic_rate_mbps = float(flow_size_bits) / (float(iat) * iat_scale_factor * 1e6)
                    total_traffic_rate_mbps += traffic_rate_mbps
        
    # Compute background load percentage
    host_to_switch_bw = host_to_switch_bw * 1e3  # Convert to Mbps
    switch_to_switch_bw = switch_to_switch_bw * 1e3  # Convert to Mbps
    total_capacity_mbps = 32 * switch_to_switch_bw + n_hosts * host_to_switch_bw
    print("No. Hosts", n_hosts)
    print("Host to Switch BW: ", host_to_switch_bw*1e-3, "Gbps")
    print("Switch to Switch BW: ", switch_to_switch_bw*1e-3, "Gbps")
    print("Total Workload Rate: ", total_traffic_rate_mbps*1e-3, "Gbps")
    print("Total Network Capacity: ", total_capacity_mbps*1e-3, "Gbps")
    background_load_percentage = (total_traffic_rate_mbps / total_capacity_mbps) * 100
    
    return background_load_percentage

# Example usage
# Construct the file paths based on the specified format
db_params = {
    "server_idx": "server_idx",
    "inter_arrival_times": "inter_arrival_time",
    "flow_sizes": "flow_size",
    "flow_ids": "flow_ids"
}

# filename = 'cache_background_inter_arrival_time_db_1.000000_flowmult_11.850000_intermult_0.db'

background_load = compute_background_load(db_params, n_hosts=100, host_to_switch_bw=1, switch_to_switch_bw=1)
print(f"Background Load: {background_load:.2f}%")

No. Hosts 100
Host to Switch BW:  1.0 Gbps
Switch to Switch BW:  1.0 Gbps
Total Workload Rate:  984.3705148727466 Gbps
Total Network Capacity:  132.0 Gbps
Background Load: 745.74%


In [46]:
def load_background_data(n_hosts):
    db_paths = {
        aspect: f"data/distributions/cache_background_{db_name}_db_{1:.6f}_flowmult_{11.85:.6f}_intermult_0_{n_hosts}_servers.db"
        for aspect, db_name in db_params.items()
    }

    dataframes = {}

    # Process each database file
    for aspect, db_path in db_paths.items():
        # Connect to the database
        conn = sqlite3.connect(db_path)
        
        # Extract the table name (assuming a single table in each db)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        table_name = cursor.fetchone()[0]
        # print(table_name)
        
        # Load the table into a DataFrame
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        dataframes[aspect] = df

        # Display first few rows
        # print(f"{aspect} data:")
        # print(df.head())
        
        # Close the connection
        conn.close()
    return dataframes

In [ ]:
def scale_iat_for_background_load(flow_sizes, iats, total_capacity_gbps, target_background_load):
    """
    Scale inter-arrival times (IATs) to achieve the target background load percentage.

    Parameters:
        flow_sizes (pd.DataFrame): Flow sizes (bytes) for each server.
        iats (pd.DataFrame): Original inter-arrival times (seconds) for each server.
        total_capacity_gbps (float): Total network capacity in Gbps.
        target_background_load (float): Desired background load as a percentage (e.g., 50%).

    Returns:
        pd.DataFrame: Scaled inter-arrival times to achieve the target background load.
    """
    # Convert total capacity to Mbps
    total_capacity_mbps = total_capacity_gbps * 1e3

    # Compute the current total traffic rate (Mbps)
    current_traffic_rate_mbps = 0
    for server in flow_sizes.columns:
        for flow_size, iat in zip(flow_sizes[server], iats[server]):
            if flow_size and iat:  # Avoid missing values
                flow_size_mbps = (int(flow_size) * 8) / 1e6  # Convert flow size to Mbps
                if float(iat) > 0:
                    current_traffic_rate_mbps += float(flow_size_mbps) / float(iat)

    # Calculate the target total traffic rate (Mbps) based on the desired load
    target_traffic_rate_mbps = (target_background_load / 100) * total_capacity_mbps

    # Determine the scaling factor for IATs
    scaling_factor = current_traffic_rate_mbps / target_traffic_rate_mbps

    # Scale the IATs
    for server in iats.columns:
        iats[server] = iats[server].apply(lambda iat: float(iat) * scaling_factor if iat else iat)

    return iats, scaling_factor

# Example usage
n_hosts = 100
flow_data = load_background_data(n_hosts)
total_cap = 32 * 1 + n_hosts * 1
iat, scaling_factor = scale_iat_for_background_load(flow_data['flow_sizes'], flow_data['inter_arrival_times'],
                    total_capacity_gbps=total_cap, target_background_load=50)

# Verification
new_background_load = compute_background_load(db_params, n_hosts=n_hosts, iat_scale_factor=scaling_factor, host_to_switch_bw=1, switch_to_switch_bw=1)
print(f"New Background Load: {new_background_load:.2f}%")

No. Hosts 100
Host to Switch BW:  1.0 Gbps
Switch to Switch BW:  1.0 Gbps
Total Workload Rate:  65.99999999999993 Gbps
Total Network Capacity:  132.0 Gbps
New Background Load: 50.00%
